### Preparation

In [1]:
import pickle
import pandas as pd
df = pd.read_csv("dataset/data_test.csv", index_col=[0])
df.shape

(202937, 11)

In [2]:
def to_sec(s):
    assert isinstance(s, str)
    r = tuple(int(x) for x in s.split(':'))
    assert len(r) == 3
    hours, mins, secs = r
    return hours*3600 + mins*60 + secs

df['time_entry_secs'] = df['time_entry'].apply(to_sec)
df['time_exit_secs'] = df['time_exit'].apply(to_sec)
df['time_delta'] = df['time_exit_secs'] - df['time_entry_secs']

center_of_x = (3750901.5068+3770901.5068)/2
center_of_y = (-19268905.6133-19208905.6133)/2

df['x_entry_delta'] = df['x_entry'].apply(lambda x: x - center_of_x)
df['y_entry_delta'] = df['y_entry'].apply(lambda y: y - center_of_y)

In [3]:
df

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,time_entry_secs,time_exit_secs,time_delta,x_entry_delta,y_entry_delta
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,11:43:17,11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,42197,42617,420,12511.970543,140624.720756
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,12:21:37,12:21:37,0.0,0.0,0.0,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,44497,44497,0,12297.123926,95364.415663
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,12:34:27,13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,45267,47651,2384,2858.344302,25490.593922
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,13:25:33,13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,48333,49393,1060,12483.027476,125461.727448
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,54212,54632,420,12216.973893,94006.259110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202932,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_3,11:23:33,11:23:33,NaN,NaN,NaN,3.762713e+06,-1.935493e+07,3.762713e+06,-1.935493e+07,41013,41013,0,1811.941088,-116028.078336
202933,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_4,12:12:10,12:12:10,NaN,NaN,NaN,3.761040e+06,-1.935274e+07,3.761040e+06,-1.935274e+07,43930,43930,0,138.809142,-113836.125316
202934,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_5,13:08:14,13:12:01,NaN,NaN,NaN,3.762680e+06,-1.935570e+07,3.762683e+06,-1.935529e+07,47294,47521,227,1778.545241,-116793.698923
202935,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_6,14:14:36,14:14:36,NaN,NaN,NaN,3.761776e+06,-1.935772e+07,3.761776e+06,-1.935772e+07,51276,51276,0,874.630976,-118812.591803


`hash` dan `trajectory_id` akan saya pisah terlebih dahulu untuk di-concat kemudian. Begitu juga dengan `time_exit`, `x_exit` dan juga `y_exit`.

In [4]:
df_hash_id = df[['hash', 'trajectory_id', 'time_exit']]
df_true_target = df[['x_exit', 'y_exit']]

Lalu, beberapa fitur saya ambil sesuai dengan keperluan untuk memprediksi.

In [5]:
df_for_predict = df[['x_entry','y_entry', 'time_entry_secs','time_exit_secs','time_delta', 'x_entry_delta', 'y_entry_delta']]

In [6]:
with open('final_model.sav', 'rb') as file:  
    pk_model = pickle.load(file)

predict_result = pk_model.predict(df_for_predict)
df_predict_result = pd.DataFrame(predict_result, columns=['x_exit_predict', 'y_exit_predict'])
df_predict_result

,x_exit_predict,y_exit_predict
0,3772510.00,-19120098.0
1,3773027.00,-19145068.0
2,3760790.25,-19222030.0
3,3769905.50,-19162014.0
4,3773117.00,-19146624.0
...,...,...
202932,3762785.75,-19354414.0
202933,3761141.75,-19354230.0
202934,3762721.50,-19353234.0
202935,3761464.00,-19356956.0


Kemudian, semua `df` yang terpisah, akan disatukan kembali dengan hasil prediksi menjadi dataframe yang baru.

In [7]:
new_df = pd.concat([df_hash_id, df_for_predict, df_true_target, df_predict_result], axis=1)
new_df

,hash,trajectory_id,time_exit,x_entry,y_entry,time_entry_secs,time_exit_secs,time_delta,x_entry_delta,y_entry_delta,x_exit,y_exit,x_exit_predict,y_exit_predict
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,11:50:17,3.773413e+06,-1.909828e+07,42197,42617,420,12511.970543,140624.720756,3.773111e+06,-1.914508e+07,3772510.00,-19120098.0
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,12:21:37,3.773199e+06,-1.914354e+07,44497,44497,0,12297.123926,95364.415663,3.773199e+06,-1.914354e+07,3773027.00,-19145068.0
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,13:14:11,3.763760e+06,-1.921342e+07,45267,47651,2384,2858.344302,25490.593922,3.771757e+06,-1.911092e+07,3760790.25,-19222030.0
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,13:43:13,3.773385e+06,-1.911344e+07,48333,49393,1060,12483.027476,125461.727448,3.773131e+06,-1.914465e+07,3769905.50,-19162014.0
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:10:32,3.773118e+06,-1.914490e+07,54212,54632,420,12216.973893,94006.259110,NaN,NaN,3773117.00,-19146624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202932,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_3,11:23:33,3.762713e+06,-1.935493e+07,41013,41013,0,1811.941088,-116028.078336,3.762713e+06,-1.935493e+07,3762785.75,-19354414.0
202933,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_4,12:12:10,3.761040e+06,-1.935274e+07,43930,43930,0,138.809142,-113836.125316,3.761040e+06,-1.935274e+07,3761141.75,-19354230.0
202934,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_5,13:12:01,3.762680e+06,-1.935570e+07,47294,47521,227,1778.545241,-116793.698923,3.762683e+06,-1.935529e+07,3762721.50,-19353234.0
202935,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_6,14:14:36,3.761776e+06,-1.935772e+07,51276,51276,0,874.630976,-118812.591803,3.761776e+06,-1.935772e+07,3761464.00,-19356956.0


Di tahap ini, sudah dipediksi semua `x_exit` dan `y_exit`, entah yang null ataupun yang tidak null.

### Check and operate on null `x_exit` and `y_exit`

Ambil semua row yang `x_exit` dan `y_exit`nya bernilai null.

In [8]:
null_df = new_df[new_df['x_exit'].isna()]
null_df

,hash,trajectory_id,time_exit,x_entry,y_entry,time_entry_secs,time_exit_secs,time_delta,x_entry_delta,y_entry_delta,x_exit,y_exit,x_exit_predict,y_exit_predict
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:10:32,3.773118e+06,-1.914490e+07,54212,54632,420,12216.973893,94006.259110,NaN,NaN,3773117.00,-19146624.0
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:30:56,3.769978e+06,-1.934136e+07,55749,55856,107,9076.651057,-102459.301053,NaN,NaN,3770153.00,-19339046.0
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,3.757468e+06,-1.923860e+07,55568,55568,0,-3433.433318,309.205438,NaN,NaN,3757326.50,-19237954.0
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:42:05,3.760505e+06,-1.935500e+07,56118,56525,407,-396.637609,-116097.676428,NaN,NaN,3760549.25,-19346738.0
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,15:05:14,3.766319e+06,-1.917013e+07,53647,54314,667,5417.579395,68777.599928,NaN,NaN,3766464.25,-19170490.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202899,ffe98f6e0adf12f9c7b51c4e9607a87a_15,traj_ffe98f6e0adf12f9c7b51c4e9607a87a_15_13,15:36:59,3.771512e+06,-1.912768e+07,56219,56219,0,10610.633640,111225.666892,NaN,NaN,3771641.00,-19126170.0
202913,fff607ecd3f8d3dcb65791e8b4c22a5f_3,traj_fff607ecd3f8d3dcb65791e8b4c22a5f_3_25,15:07:30,3.755978e+06,-1.926623e+07,53695,54450,755,-4924.001300,-27327.756456,NaN,NaN,3755735.25,-19267600.0
202914,fff813b56230c2f026f783f5b9f9ca90_19,traj_fff813b56230c2f026f783f5b9f9ca90_19_0,15:08:02,3.751181e+06,-1.914034e+07,54482,54482,0,-9720.758158,98562.242727,NaN,NaN,3750904.00,-19140498.0
202929,fff9400843a88c3bfe52e7ce8bf97316_19,traj_fff9400843a88c3bfe52e7ce8bf97316_19_17,15:19:14,3.768946e+06,-1.914828e+07,54823,55154,331,8044.719378,90626.455494,NaN,NaN,3769029.50,-19149818.0


Pastikan semua `x_exit` dan`y_exit` yang bernilai null bernilai 15 pada `time_exit_in_hour`nya atau tidak lebih dari `16:00:00` pada `time_exit`nya.

In [9]:
to_hour = lambda s: s[:2]
null_df['time_exit_in_hour'] = null_df['time_exit'].apply(to_hour)
null_df['time_exit_in_hour'].unique()

<ipython-input-9-03c89f8f0d64>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df['time_exit_in_hour'] = null_df['time_exit'].apply(to_hour)


array(['15', '16'], dtype=object)

In [10]:
null_df[null_df['time_exit_in_hour']=='16']

,hash,trajectory_id,time_exit,x_entry,y_entry,time_entry_secs,time_exit_secs,time_delta,x_entry_delta,y_entry_delta,x_exit,y_exit,x_exit_predict,y_exit_predict,time_exit_in_hour
5067,0663e04490728c65b7498e7b6490cedb_1,traj_0663e04490728c65b7498e7b6490cedb_1_7,16:00:00,3.774878e+06,-1.920461e+07,56999,57600,601,13976.935042,34300.586137,NaN,NaN,3773338.25,-19195518.0,16
7258,090c486a26325217e649784bf288b401_11,traj_090c486a26325217e649784bf288b401_11_9,16:00:00,3.768539e+06,-1.923017e+07,57600,57600,0,7637.290041,8732.959773,NaN,NaN,3768495.75,-19230152.0,16
13253,108bb32346224aba3b37695f4c02ec36_11,traj_108bb32346224aba3b37695f4c02ec36_11_11,16:00:00,3.751079e+06,-1.935127e+07,57060,57600,540,-9822.058895,-112363.646761,NaN,NaN,3750331.50,-19341398.0,16
16421,1468d31ac74f6e78c0cab95c97ff1fe4_15,traj_1468d31ac74f6e78c0cab95c97ff1fe4_15_6,16:00:00,3.744082e+06,-1.921297e+07,57600,57600,0,-16819.602086,25933.083093,NaN,NaN,3744860.50,-19211344.0,16
24047,1ddc169a4d5eaadd457081190d79f830_3,traj_1ddc169a4d5eaadd457081190d79f830_3_5,16:00:00,3.753625e+06,-1.910525e+07,57600,57600,0,-7276.182140,133655.760217,NaN,NaN,3753914.75,-19107610.0,16
55056,456ac4c877a714c040a065b5b747f5b4_29,traj_456ac4c877a714c040a065b5b747f5b4_29_9,16:00:00,3.761341e+06,-1.910805e+07,57437,57600,163,439.371767,130854.891931,NaN,NaN,3761290.50,-19106170.0,16
116732,930bc7b1e3226defaafb27e2a4ec3164_23,traj_930bc7b1e3226defaafb27e2a4ec3164_23_9,16:00:00,3.757730e+06,-1.925221e+07,57600,57600,0,-3171.832515,-13306.024790,NaN,NaN,3757778.50,-19251676.0,16
160819,cab9292f603d64902d9f2d1fab797e89_25,traj_cab9292f603d64902d9f2d1fab797e89_25_2,16:00:00,3.773095e+06,-1.914349e+07,57600,57600,0,12193.596800,95420.531644,NaN,NaN,3773067.75,-19145026.0,16
202720,ffac2b0a22b66c3f1893f1655b63cbd6_19,traj_ffac2b0a22b66c3f1893f1655b63cbd6_19_3,16:00:00,3.770914e+06,-1.910792e+07,57600,57600,0,10012.847975,130988.825844,NaN,NaN,3771202.00,-19111250.0,16


Setelah pengecekan selesai, lalu kita aplikasikan inside/outside rule berdasarkan `x_exit_predict` dan `y_exit_predict`. 

In [11]:
inside_area = lambda x,y: 3750901.5068<=x<=3770901.5068 and -19268905.6133<=y<=-19208905.6133
is_in_area = lambda args: int(inside_area(*args))

null_df['city_center'] = null_df[['x_exit_predict', 'y_exit_predict']].apply(is_in_area, axis=1)
null_df

<ipython-input-11-a70c690cd03a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df['city_center'] = null_df[['x_exit_predict', 'y_exit_predict']].apply(is_in_area, axis=1)


,hash,trajectory_id,time_exit,x_entry,y_entry,time_entry_secs,time_exit_secs,time_delta,x_entry_delta,y_entry_delta,x_exit,y_exit,x_exit_predict,y_exit_predict,time_exit_in_hour,city_center
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:10:32,3.773118e+06,-1.914490e+07,54212,54632,420,12216.973893,94006.259110,NaN,NaN,3773117.00,-19146624.0,15,0
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:30:56,3.769978e+06,-1.934136e+07,55749,55856,107,9076.651057,-102459.301053,NaN,NaN,3770153.00,-19339046.0,15,0
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,3.757468e+06,-1.923860e+07,55568,55568,0,-3433.433318,309.205438,NaN,NaN,3757326.50,-19237954.0,15,1
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:42:05,3.760505e+06,-1.935500e+07,56118,56525,407,-396.637609,-116097.676428,NaN,NaN,3760549.25,-19346738.0,15,0
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,15:05:14,3.766319e+06,-1.917013e+07,53647,54314,667,5417.579395,68777.599928,NaN,NaN,3766464.25,-19170490.0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202899,ffe98f6e0adf12f9c7b51c4e9607a87a_15,traj_ffe98f6e0adf12f9c7b51c4e9607a87a_15_13,15:36:59,3.771512e+06,-1.912768e+07,56219,56219,0,10610.633640,111225.666892,NaN,NaN,3771641.00,-19126170.0,15,0
202913,fff607ecd3f8d3dcb65791e8b4c22a5f_3,traj_fff607ecd3f8d3dcb65791e8b4c22a5f_3_25,15:07:30,3.755978e+06,-1.926623e+07,53695,54450,755,-4924.001300,-27327.756456,NaN,NaN,3755735.25,-19267600.0,15,1
202914,fff813b56230c2f026f783f5b9f9ca90_19,traj_fff813b56230c2f026f783f5b9f9ca90_19_0,15:08:02,3.751181e+06,-1.914034e+07,54482,54482,0,-9720.758158,98562.242727,NaN,NaN,3750904.00,-19140498.0,15,0
202929,fff9400843a88c3bfe52e7ce8bf97316_19,traj_fff9400843a88c3bfe52e7ce8bf97316_19_17,15:19:14,3.768946e+06,-1.914828e+07,54823,55154,331,8044.719378,90626.455494,NaN,NaN,3769029.50,-19149818.0,15,0


Setelah ini, ambil semua kolom yang diperlukan.

In [12]:
sample_with_predicted_coordinate = null_df[['trajectory_id', 'x_exit_predict', 'y_exit_predict', 'city_center']]

In [13]:
final_sample = null_df[['trajectory_id', 'city_center']]

Saya akan buat 2 buah export dari `sample prediction result`.
- Yang pertama, dengan predicted coordinate.
- Yang kedua, hanya `trajectory_id` dan klasifikasi inside/outside (final sample)

In [14]:
sample_with_predicted_coordinate.to_csv("sample_with_predicted_coordinate.csv", index=False)
final_sample.to_csv("sample.csv", index=False)